## Smart RAG
- Know what you know and what you don't know.


In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [2]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [5]:

solar_summary = """
SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling

We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

In [6]:
# Tools
from langchain_core.tools import tool
import requests
import os
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


@tool
def solar_paper_search(query: str) -> str:
    """Query for research paper about solarllm, dus, llm and general AI.
    If the query is about DUS, Upstage, AI related topics, use this.
    """
    return solar_summary


@tool
def internet_search(query: str) -> str:
    """This is for query for internet search engine like Google.
    Query for general topics.
    """
    return tavily.search(query=query)


@tool
def get_news(topic: str) -> str:
    """Get latest news about a topic.
    If users are more like recent news, use this.
    """
    # https://newsapi.org/v2/everything?q=tesla&from=2024-04-01&sortBy=publishedAt&apiKey=API_KEY
    # change this to request news from a real API
    news_url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={os.environ['NEWS_API_KEY']}"
    respnse = requests.get(news_url)
    return respnse.json()


tools = [solar_paper_search, internet_search, get_news]


llm_with_tools = llm.bind_tools(tools)

In [7]:
def call_tool(tool_call):
    tool_name = tool_call["name"].lower()
    if tool_name not in globals():
        print("Tool not found", tool_name)
        return None
    selected_tool = globals()[tool_name]
    return selected_tool.invoke(tool_call["args"])

In [8]:
llm_with_tools.invoke("What is Solar LLM?").tool_calls


[]

In [9]:
llm_with_tools.invoke("What is news about Seoul").tool_calls


[{'name': 'get_news',
  'args': {'topic': 'Seoul'},
  'id': '27afeab7-c37e-4620-b09d-9df4124a0e59'}]

In [10]:
llm_with_tools.invoke("What's best place in Seoul?").tool_calls


[{'name': 'internet_search',
  'args': {'query': 'Best place in Seoul'},
  'id': 'bda41077-04a6-4e9d-b9e0-1bc4ab8118a3'}]

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer for question from the following context. 
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [12]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def tool_rag(question):
    tool_calls = llm_with_tools.invoke(question).tool_calls
    print(tool_calls)
    context = ""
    for tool_call in tool_calls:
        context += str(call_tool(tool_call))

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [13]:
tool_rag("What is Solar llm?")

[]


'Solar Illumination is a company that develops and manufactures advanced solar energy systems for residential and commercial use. Their products include solar panels, solar batteries, and solar energy management systems, all designed to provide clean, renewable energy to homes and businesses.'

In [14]:
tool_rag("What is news about Tesla?")

[{'name': 'get_news', 'args': {'topic': 'Tesla'}, 'id': '215039f2-9ee6-41b9-a756-6284a9d0a86b'}]


'Here are the news about Tesla based on the provided context:\n\n1. Tesla has issued a voluntary recall for nearly 4,000 Cybertrucks due to a potential issue with the accelerator pedal.\n2. Federal regulators concluded that Tesla\'s Autopilot system lacked standard protections in an investigative report into crashes and deaths associated with the technology.\n3. Tesla announced plans to launch "more affordable vehicles" and a "Cybercab" before mid-2025, sooner than originally planned.\n4. Tesla has recalled all 3,878 Cybertrucks due to a faulty accelerator pedal.\n5. Elon Musk is reportedly laying off hundreds more employees, including the majority of the Supercharging team.\n6. Some Tesla Cybertruck buyers have reported delays in deliveries, with some claiming that an accelerator issue is to blame.\n7. Tesla announced that it will lay off more than 10% of its global workforce.\n8. Climate protestors broke through police barricades on Friday near a Tesla gigafactory in Germany.\n9. The

In [15]:
tool_rag("iPhone 13 spec?")

[{'name': 'internet_search', 'args': {'query': 'iPhone 13 spec'}, 'id': '6b7a00ee-2d33-410f-a727-a15733e28f8b'}]


'The iPhone 13 specs include:\n\n* Display: 6.1 inches with a resolution of 2532 x 1170 pixels and a 60Hz refresh rate\n* Camera: 12 MP dual camera and a 12 MP front camera\n* Hardware: Apple A15 Bionic chipset, 4GB RAM\n* Storage: 128GB, not expandable\n* Battery: 3240 mAh\n* Size: 2.82 inches x 5.78 inches x 0.30 inches\n* Weight: 6.14 ounces (174 grams)\n\nThe iPhone 13 is available in the following colors: (PRODUCT)RED, Starlight, Midnight, Blue, Pink, and Green. It has a Ceramic Shield front and a glass back and aluminum design.'

# Excercise
Solar LLM is small, so it might not give the best results for complex tasks. For those, you can use larger LLMs like GPT-4. However, for quick answers and summaries, using a small size LLM like Solar can give better performance and efficiency.

Please note that good engineering involves making things work with limited components.
![Engineering](figures/engineering.jpg)